In [1]:
from dotenv import load_dotenv
import os
from supabase import create_client
import fonctions.fonctions_api as f
import fonctions.fonctions_tableaux as ft

C:\Users\guill\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()  # charge le .env
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
season = os.getenv("SEASON")
supabase = create_client(url, key)

In [22]:
f.acheter_joueur(supabase, id_user=4, id_contrat=368)

INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Deadline?select=START%2CEND&START=lte.2025-05-03T12%3A19%3A19.894629%2B02%3A00&END=gte.2025-05-03T12%3A19%3A19.894629%2B02%3A00 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&id_contrat=eq.368&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Possession?select=id_possession&id_user=eq.4&id_contrat=eq.368&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Possession?select=id_possession&id_user=eq.4&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Valeur_Actuelle?select=valeur%2Cdate&id_contrat=eq.368&order=date.desc&limit=1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Banque?select=solde%2Cdatetime&id_user=eq.4&order=datetim

✅ Achat effectué : joueur 368 pour 12.25 crédits. Nouveau solde : 1.25


In [25]:
ft.afficher_effectif(supabase,id_user=4)

INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Possession?select=id_contrat&id_user=eq.4&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Valeur_Actuelle?select=valeur&id_contrat=eq.323&order=date.desc&limit=1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Transaction?select=prix&id_contrat=eq.323&id_user=eq.4&type_transaction=eq.True&order=datetime.desc&limit=1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_joueur%2Cid_equipe&id_contrat=eq.323 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Joueur?select=nom%2Cprenom&id_joueur=eq.P012712+++ "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Equipe?select=nom&id_equipe=eq.PAN "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.sup

[{'id_contrat': 323,
  'Joueur': 'JUANCHO HERNANGOMEZ',
  'Équipe': 'Panathinaikos AKTOR Athens',
  'Valeur actuelle': 16.25,
  'Prix d’achat': 16.25,
  'Dernier match': '2025-05-02T19:30:00+00:00',
  'Dernier PER': 18},
 {'id_contrat': 360,
  'Joueur': 'JUSTIN ANDERSON',
  'Équipe': 'FC Barcelona',
  'Valeur actuelle': 7.75,
  'Prix d’achat': 7.75,
  'Dernier match': '2025-05-02T20:45:00+00:00',
  'Dernier PER': 12},
 {'id_contrat': 320,
  'Joueur': 'KENDRICK NUNN',
  'Équipe': 'Panathinaikos AKTOR Athens',
  'Valeur actuelle': 13,
  'Prix d’achat': 13,
  'Dernier match': '2025-05-02T19:30:00+00:00',
  'Dernier PER': 3},
 {'id_contrat': 493,
  'Joueur': 'DARIUS THOMPSON',
  'Équipe': 'Anadolu Efes Istanbul',
  'Valeur actuelle': 7.5,
  'Prix d’achat': 7.5,
  'Dernier match': '2025-05-02T19:30:00+00:00',
  'Dernier PER': 14},
 {'id_contrat': 596,
  'Joueur': 'DANIEL THEIS',
  'Équipe': 'AS Monaco',
  'Valeur actuelle': 10.5,
  'Prix d’achat': 10.5,
  'Dernier match': '2025-05-02T20:45:

In [26]:
id_user = 4

In [28]:
contrats_actifs_res

APIResponse[~_ReturnT](data=[{'id_contrat': 315}, {'id_contrat': 316}, {'id_contrat': 317}, {'id_contrat': 318}, {'id_contrat': 319}, {'id_contrat': 320}, {'id_contrat': 321}, {'id_contrat': 322}, {'id_contrat': 323}, {'id_contrat': 324}, {'id_contrat': 325}, {'id_contrat': 326}, {'id_contrat': 327}, {'id_contrat': 328}, {'id_contrat': 329}, {'id_contrat': 330}, {'id_contrat': 331}, {'id_contrat': 332}, {'id_contrat': 333}, {'id_contrat': 334}, {'id_contrat': 335}, {'id_contrat': 359}, {'id_contrat': 360}, {'id_contrat': 361}, {'id_contrat': 362}, {'id_contrat': 363}, {'id_contrat': 364}, {'id_contrat': 365}, {'id_contrat': 366}, {'id_contrat': 367}, {'id_contrat': 368}, {'id_contrat': 431}, {'id_contrat': 432}, {'id_contrat': 433}, {'id_contrat': 434}, {'id_contrat': 435}, {'id_contrat': 436}, {'id_contrat': 437}, {'id_contrat': 438}, {'id_contrat': 439}, {'id_contrat': 440}, {'id_contrat': 441}, {'id_contrat': 442}, {'id_contrat': 443}, {'id_contrat': 444}, {'id_contrat': 445}, {'id_

In [ ]:
def afficher_joueurs_disponibles(supabase, id_user: int):
    # 1. Récupérer tous les contrats actifs
    contrats_actifs_res = supabase.table("Contrat") \
        .select("id_contrat") \
        .is_("END", None) \
        .execute()

    if not contrats_actifs_res.data:
        return []

    # 2. Récupérer les contrats déjà possédés par l'utilisateur
    possedes_res = supabase.table("Possession") \
        .select("id_contrat") \
        .eq("id_user", id_user) \
        .is_("END", None) \
        .execute()

    contrats_possedes = {p["id_contrat"] for p in possedes_res.data} if possedes_res.data else set()

    # 3. Filtrer les contrats disponibles
    contrats_disponibles = [
        contrat["id_contrat"]
        for contrat in contrats_actifs_res.data
        if contrat["id_contrat"] not in contrats_possedes
    ]

    perf_res = supabase.table("vue_tableau_recap") \
        .select("id_contrat, date, PER, nom,prenom,nom_equipe") \
        .in_("id_contrat", contrats_disponibles) \
        .eq("rang", 1) \
        .execute()


    # Étape 1 : extraire les id_contrat
    id_contrats = [row["id_contrat"] for row in perf_res.data]

    # Étape 2 : récupérer les valeurs actuelles les plus récentes
    valeurs_res = supabase.table("Valeur_Actuelle") \
        .select("id_contrat, valeur") \
        .in_("id_contrat", id_contrats) \
        .order("date", desc=True) \
        .execute()

    # Étape 3 : garder la valeur la plus récente par contrat
    valeurs_dict = {}
    for row in valeurs_res.data:
        contrat = row["id_contrat"]
        if contrat not in valeurs_dict:
            valeurs_dict[contrat] = row["valeur"]

    effectif = []
    # Étape 4 : enrichir perf_res
    for row in perf_res.data:
        row["valeur_actuelle"] = valeurs_dict.get(row["id_contrat"], "?")
        effectif.append({
            "id_contrat": row["id_contrat"],
            "Joueur": f"{row['prenom']} {row['nom']}",
            "Équipe": row["nom_equipe"],
            "Valeur actuelle":  row["valeur_actuelle"],
            "Dernier match": row["date"],
            "Dernier PER": row["PER"]
        })
    return effectif

INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Possession?select=id_contrat&id_user=eq.4&END=is.null "HTTP/2 200 OK"


INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/vue_tableau_recap?select=id_contrat%2Cdate%2CPER%2Cnom%2Cprenom%2Cnom_equipe&id_contrat=in.%28315%2C317%2C318%2C319%2C321%2C322%2C324%2C325%2C326%2C327%2C328%2C329%2C330%2C331%2C332%2C333%2C334%2C361%2C362%2C363%2C364%2C365%2C366%2C367%2C431%2C432%2C433%2C434%2C435%2C436%2C437%2C438%2C439%2C440%2C441%2C442%2C443%2C444%2C445%2C446%2C447%2C448%2C449%2C450%2C452%2C453%2C533%2C534%2C540%2C541%2C545%2C546%2C555%2C559%2C570%2C571%2C572%2C574%2C575%2C578%2C579%2C580%2C583%2C584%2C588%2C589%2C595%2C597%2C606%2C488%2C489%2C490%2C491%2C492%2C494%2C495%2C496%2C497%2C498%2C499%2C500%2C503%2C504%2C506%2C515%2C516%2C519%2C520%2C522%2C599%2C601%29&rang=eq.1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Valeur_Actuelle?select=id_contrat%2Cvaleur&id_contrat=in.%28315%2C317%2C318%2C319%2C321%2C322%2C324%2C325%2C326%2C327%2C328%2C329%2C330%2C331%2C332%2C333%2C334%2C361%2C362%2C363

[{'id_contrat': 315,
  'Joueur': 'PANAGIOTIS KALAITZAKIS',
  'Équipe': 'Panathinaikos AKTOR Athens',
  'Valeur actuelle': 1.25,
  'Dernier match': '2025-04-24T20:30:00',
  'Dernier PER': 2},
 {'id_contrat': 317,
  'Joueur': 'KOSTAS SLOUKAS',
  'Équipe': 'Panathinaikos AKTOR Athens',
  'Valeur actuelle': 15.5,
  'Dernier match': '2025-05-02T19:30:00',
  'Dernier PER': 26},
 {'id_contrat': 318,
  'Joueur': 'CEDI OSMAN',
  'Équipe': 'Panathinaikos AKTOR Athens',
  'Valeur actuelle': 13,
  'Dernier match': '2025-05-02T19:30:00',
  'Dernier PER': 19},
 {'id_contrat': 319,
  'Joueur': 'JERIAN GRANT',
  'Équipe': 'Panathinaikos AKTOR Athens',
  'Valeur actuelle': 8.75,
  'Dernier match': '2025-05-02T19:30:00',
  'Dernier PER': 12},
 {'id_contrat': 321,
  'Joueur': 'MATHIAS LESSORT',
  'Équipe': 'Panathinaikos AKTOR Athens',
  'Valeur actuelle': 17.75,
  'Dernier match': '2024-12-19T18:45:00',
  'Dernier PER': 21},
 {'id_contrat': 322,
  'Joueur': 'MARIUS GRIGONIS',
  'Équipe': 'Panathinaikos 

In [ ]:
# f.acheter_joueur(supabase, id_user = 4, id_contrat = 323)
# f.acheter_joueur(supabase, id_user = 4, id_contrat = 360)
# f.acheter_joueur(supabase, id_user = 4, id_contrat = 435)
# f.acheter_joueur(supabase, id_user = 4, id_contrat = 328)

In [ ]:
# f.ajouter_user(supabase, pseudo = "gpain1999", mot_de_passe = "Iledefrance75@", adresse_mail = "guillaume.pain9@gmail.com")

In [ ]:
# f.finir_contrats_equipe(supabase, id_equipe = "ZAL")

In [ ]:
from euroleague_api.game_stats import GameStats

gs = GameStats()

ggs = gs.get_game_report(season=season, game_code=324)

In [ ]:
# f.get_update_match_data(supabase, season)